In [1]:
# ========== CONFIGURATION ==========
MODEL_VERSION = "single"  # Options: "single" or "two_step"

TRAIN_PATH = '/home/stargix/Desktop/hackathons/datathon/train/train'
TEST_PATH = '/home/stargix/Desktop/hackathons/datathon/test/test'
TARGET_COL = "iap_revenue_d7"
TRAIN_SAMPLE_FRAC = 0.10  # Adjust for more/less data

print(f"Selected model: {MODEL_VERSION}")

Selected model: single


In [2]:
import numpy as np
import pandas as pd
import dask
import dask.dataframe as dd
from lightgbm import LGBMRegressor, LGBMClassifier
from sklearn.metrics import mean_squared_log_error
import gc
import os

dask.config.set({"dataframe.convert-string": False})

## Helper Functions

In [ ]:
def preprocess_train_valid(X_train, X_valid, num_cols, cat_cols):
    """Preprocess train and validation sets with 0-filling and normalization."""
    from sklearn.preprocessing import StandardScaler
    
    X_train = X_train.copy()
    X_valid = X_valid.copy()
    
    # Numeric: fill NaN with 0 for missing values
    if num_cols:
        print(f"  Filling {len(num_cols)} numeric columns with 0 for missing values...")
        for c in num_cols:
            X_train[c] = X_train[c].fillna(0)
            X_valid[c] = X_valid[c].fillna(0)
        
        # Normalization: StandardScaler for numeric features
        print(f"  Applying StandardScaler normalization...")
        scaler = StandardScaler()
        X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
        X_valid[num_cols] = scaler.transform(X_valid[num_cols])
    else:
        scaler = None
    
    # Categorical: convert to strings and set categories based on train
    for c in cat_cols:
        X_train[c] = X_train[c].astype("object").fillna("unknown").astype(str)
        X_train[c] = X_train[c].astype("category")
        
        cats = X_train[c].cat.categories
        X_valid[c] = X_valid[c].astype("object").fillna("unknown").astype(str)
        X_valid[c] = X_valid[c].astype(pd.api.types.CategoricalDtype(categories=cats))
    
    return X_train, X_valid, scaler

Helper functions loaded (with advanced preprocessing).


## Load and Prepare Data

In [4]:
from glob import glob
import os

# Train: Oct 1-5, Valid: Oct 6
filters_train = [("datetime", ">=", "2025-10-01-00-00"),
                 ("datetime", "<",  "2025-10-06-00-00")]
filters_valid = [("datetime", ">=", "2025-10-06-00-00"),
                 ("datetime", "<",  "2025-10-07-00-00")]

# Obtener lista de archivos parquet
parquet_files_all = glob(os.path.join(TRAIN_PATH, '**/part-*.parquet'), recursive=True)

# 🔥 REDUCIR MÁS: Solo 5-10% de archivos
num_files_train = max(1, int(len(parquet_files_all) * 0.15))  # Cambié a 5%
parquet_files_train = parquet_files_all[:num_files_train]

print(f"Using {num_files_train} out of {len(parquet_files_all)} train files")

# 🔥 DROPEAR COLUMNAS ANTES DE COMPUTE
cols_to_drop_early = IGNORE_BIG_COLS + ["row_id", "datetime"]

# Cargar TRAIN
print("Loading train data...")
dd_train = dd.read_parquet(
    parquet_files_train, 
    filters=filters_train,
    engine='pyarrow'
)

# Dropear columnas pesadas ANTES de compute
existing_cols = [c for c in cols_to_drop_early if c in dd_train.columns]
dd_train = dd_train.drop(columns=existing_cols)

# 🔥 SAMPLE EN DASK (no en pandas)
train_sample = dd_train.sample(frac=TRAIN_SAMPLE_FRAC, random_state=42).compute()
train_sample = reduce_memory(train_sample)

print(f"Train loaded: {train_sample.shape}, Memory: {train_sample.memory_usage(deep=True).sum() / 1e9:.2f} GB")

# Limpiar memoria
del dd_train
gc.collect()

# Cargar VALID (después de liberar train)
print("\nLoading validation data...")
dd_valid = dd.read_parquet(
    parquet_files_train,  # Mismos archivos
    filters=filters_valid,
    engine='pyarrow'
)

existing_cols = [c for c in cols_to_drop_early if c in dd_valid.columns]
dd_valid = dd_valid.drop(columns=existing_cols)

# 🔥 SAMPLE MENOS EN VALID (solo necesitas evaluar, no entrenar)
valid_df = dd_valid.sample(frac=min(0.5, TRAIN_SAMPLE_FRAC), random_state=42).compute()
valid_df = reduce_memory(valid_df)

print(f"Valid loaded: {valid_df.shape}, Memory: {valid_df.memory_usage(deep=True).sum() / 1e9:.2f} GB")

del dd_valid
gc.collect()

print(f"\n✓ Data loaded successfully")
print(f"Total memory: ~{(train_sample.memory_usage(deep=True).sum() + valid_df.memory_usage(deep=True).sum()) / 1e9:.2f} GB")

Using 21 out of 144 train files
Loading train data...


Train loaded: (271487, 56), Memory: 0.40 GB

Loading validation data...

Loading validation data...
Valid loaded: (28373, 56), Memory: 0.04 GB

✓ Data loaded successfully
Total memory: ~0.44 GB


In [5]:
# Extract targets
y_train = train_sample[TARGET_COL].values
y_valid = valid_df[TARGET_COL].values

# Always extract buyer labels (needed for two-step model)
y_train_buyer = train_sample["buyer_d7"].values
y_valid_buyer = valid_df["buyer_d7"].values

if MODEL_VERSION == "two_step":
    print(f"Buyer ratio in train: {y_train_buyer.mean():.4f}")
    print(f"Buyer ratio in valid: {y_valid_buyer.mean():.4f}")

# Prepare features
cols_to_drop = ["row_id", "datetime"] + LABEL_COLS
feature_cols = [c for c in train_sample.columns if c not in cols_to_drop]

X_train = train_sample[feature_cols].copy()
X_valid = valid_df[feature_cols].copy()

# Detect and remove list-like columns
listlike_cols = detect_listlike_columns(X_train, cols=feature_cols)
print(f"Removing {len(listlike_cols)} list-like columns: {listlike_cols}")
X_train = X_train.drop(columns=listlike_cols)
X_valid = X_valid.drop(columns=listlike_cols)

# Identify numeric and categorical columns
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

print(f"Features: {len(X_train.columns)} ({len(num_cols)} numeric, {len(cat_cols)} categorical)")

# Preprocess with advanced imputation and normalization
print("\nApplying preprocessing (KNN imputation + StandardScaler + categorical encoding)...")
X_train_prep, X_valid_prep, scaler = preprocess_train_valid(X_train, X_valid, num_cols, cat_cols)
print(f"Data prepared: X_train {X_train_prep.shape}, X_valid {X_valid_prep.shape}")

# Memory optimization
X_train_prep = reduce_memory(X_train_prep)
X_valid_prep = reduce_memory(X_valid_prep)
print(f"✓ Memory optimized")
print(f"✓ Scaler saved for test predictions")

Removing 14 list-like columns: ['avg_daily_sessions', 'avg_duration', 'cpm', 'cpm_pct_rk', 'ctr', 'ctr_pct_rk', 'hour_ratio', 'iap_revenue_usd_bundle', 'iap_revenue_usd_category', 'iap_revenue_usd_category_bottom_taxonomy', 'num_buys_bundle', 'num_buys_category', 'num_buys_category_bottom_taxonomy', 'rwd_prank']
Features: 26 (11 numeric, 15 categorical)

Applying preprocessing (KNN imputation + StandardScaler + categorical encoding)...
  Applying KNN imputation to 11 numeric columns...


KeyboardInterrupt: 

## Train Models

In [ ]:
MODEL_VERSION == "single"

if MODEL_VERSION == "single":
    print("=" * 50)
    print("TRAINING SINGLE LGBM REGRESSOR")
    print("=" * 50)
    
    # Transform target to log space
    y_train_log = np.log1p(y_train)
    y_valid_log = np.log1p(y_valid)
    
    # Train single regressor
    model = LGBMRegressor(
        objective="regression",
        n_estimators=600,
        learning_rate=0.05,
        num_leaves=255,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=0.0,
        verbosity=-1,
        random_state=42
    )
    
    model.fit(X_train_prep, y_train_log)
    print("✓ Model trained")
    
    # Predict
    valid_pred_log = model.predict(X_valid_prep)
    valid_pred = np.expm1(valid_pred_log)
    valid_pred = np.clip(valid_pred, 0, None)
    
    # Store for submission
    models = {"regressor": model}
    
elif MODEL_VERSION == "two_step":
    print("=" * 50)
    print("TRAINING TWO-STEP MODEL (CLASSIFIER + REGRESSOR)")
    print("=" * 50)
    
    # Step 1: Train buyer classifier
    print("\n[1/2] Training buyer classifier...")
    buyer_classifier = LGBMClassifier(
        objective="binary",
        n_estimators=400,
        learning_rate=0.05,
        num_leaves=127,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        reg_lambda=0.1,
        verbosity=-1,
        random_state=42
    )
    
    buyer_classifier.fit(X_train_prep, y_train_buyer)
    print("✓ Buyer classifier trained")
    
    # Get buyer probabilities
    buyer_prob_train = buyer_classifier.predict_proba(X_train_prep)[:, 1]
    buyer_prob_valid = buyer_classifier.predict_proba(X_valid_prep)[:, 1]
    
    # Step 2: Train revenue regressor on buyers only
    print("\n[2/2] Training revenue regressor (buyers only)...")
    buyer_mask_train = y_train > 0
    
    X_train_buyers = X_train_prep[buyer_mask_train]
    y_train_buyers_log = np.log1p(y_train[buyer_mask_train])
    
    print(f"Training on {buyer_mask_train.sum()} buyers out of {len(y_train)} samples")
    
    revenue_regressor = LGBMRegressor(
        objective="regression",
        n_estimators=600,
        learning_rate=0.05,
        num_leaves=255,
        max_depth=-1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.0,
        reg_lambda=0.0,
        verbosity=-1,
        random_state=42
    )
    
    revenue_regressor.fit(X_train_buyers, y_train_buyers_log)
    print("✓ Revenue regressor trained")
    
    # Step 3: Combined predictions
    valid_revenue_pred_log = revenue_regressor.predict(X_valid_prep)
    valid_revenue_pred = np.expm1(valid_revenue_pred_log)
    valid_revenue_pred = np.clip(valid_revenue_pred, 0, None)
    
    # Final prediction: P(buyer) * E[revenue | buyer]
    valid_pred = buyer_prob_valid * valid_revenue_pred
    
    # Store for submission
    models = {
        "classifier": buyer_classifier,
        "regressor": revenue_regressor
    }
    
    # Additional metrics
    buyer_acc = (buyer_classifier.predict(X_valid_prep) == y_valid_buyer).mean()
    print(f"\nBuyer classifier accuracy: {buyer_acc:.4f}")
    print(f"Average predicted buyer probability: {buyer_prob_valid.mean():.4f}")

else:
    raise ValueError(f"Invalid MODEL_VERSION: {MODEL_VERSION}")

TRAINING SINGLE LGBM REGRESSOR
✓ Model trained
✓ Model trained


## Evaluate

In [ ]:
# Calculate MSLE
msle_model = mean_squared_log_error(y_valid, valid_pred)
msle_baseline = mean_squared_log_error(y_valid, np.zeros_like(y_valid))

print("\n" + "=" * 50)
print("VALIDATION RESULTS")
print("=" * 50)
print(f"Model: {MODEL_VERSION.upper()}")
print(f"MSLE: {msle_model:.6f}")
print(f"Baseline (all zeros): {msle_baseline:.6f}")
print(f"Improvement: {((msle_baseline - msle_model) / msle_baseline * 100):.2f}%")

# Distribution stats
print(f"\nPrediction stats:")
print(f"  Mean: {valid_pred.mean():.4f}")
print(f"  Median: {np.median(valid_pred):.4f}")
print(f"  Max: {valid_pred.max():.4f}")
print(f"  % Non-zero: {(valid_pred > 0).mean() * 100:.2f}%")


VALIDATION RESULTS
Model: SINGLE
MSLE: 0.186644
Baseline (all zeros): 0.228072
Improvement: 18.16%

Prediction stats:
  Mean: 0.1340
  Median: 0.0010
  Max: 57.9766
  % Non-zero: 60.10%


In [ ]:
## Grid Search for Single Model

import itertools
from datetime import datetime

# Verificar que estamos en modo single
if MODEL_VERSION != "single":
    print("⚠️ Grid search only works with MODEL_VERSION='single'")
    print("Please change MODEL_VERSION in the first cell and re-run from the beginning")
else:
    print("=" * 60)
    print("GRID SEARCH - SINGLE LGBM REGRESSOR")
    print("=" * 60)
    
    # Define parameter grid
    param_grid = {
        'n_estimators': [400, 600, 800],
        'learning_rate': [0.03, 0.05, 0.07],
        'num_leaves': [127, 255, 511],
        'max_depth': [-1, 10, 15],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.7, 0.8, 0.9],
    }
    
    # Generate all combinations
    keys = param_grid.keys()
    values = param_grid.values()
    combinations = list(itertools.product(*values))
    
    print(f"\nTotal combinations: {len(combinations)}")
    print(f"Estimated time: ~{len(combinations) * 2} minutes (aprox 2 min/model)\n")
    
    # Store results
    results = []
    
    # Transform target once
    y_train_log = np.log1p(y_train)
    
    # Grid search
    for i, params in enumerate(combinations[:10], 1):  # Limit to first 10 for testing
        param_dict = dict(zip(keys, params))
        
        # 🔥 FIX: Convertir parámetros int explícitamente
        param_dict['n_estimators'] = int(param_dict['n_estimators'])
        param_dict['num_leaves'] = int(param_dict['num_leaves'])
        param_dict['max_depth'] = int(param_dict['max_depth'])
        
        print(f"[{i}/{min(10, len(combinations))}] Testing: {param_dict}")
        
        # Train model
        model = LGBMRegressor(
            objective="regression",
            reg_alpha=0.0,
            reg_lambda=0.0,
            verbosity=-1,
            random_state=42,
            **param_dict
        )
        
        start_time = datetime.now()
        model.fit(X_train_prep, y_train_log)
        train_time = (datetime.now() - start_time).total_seconds()
        
        # Predict
        valid_pred_log = model.predict(X_valid_prep)
        valid_pred = np.expm1(valid_pred_log)
        valid_pred = np.clip(valid_pred, 0, None)
        
        # Evaluate
        msle = mean_squared_log_error(y_valid, valid_pred)
        
        # Store result
        result = {
            **param_dict,
            'msle': msle,
            'train_time': train_time
        }
        results.append(result)
        
        print(f"  MSLE: {msle:.6f} | Time: {train_time:.1f}s\n")
    
    # Convert to DataFrame and sort
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('msle')
    
    print("\n" + "=" * 60)
    print("TOP 5 BEST MODELS")
    print("=" * 60)
    print(results_df.head(5).to_string(index=False))
    
    print("\n" + "=" * 60)
    print("BEST MODEL PARAMETERS")
    print("=" * 60)
    best_params = results_df.iloc[0].to_dict()
    for key, value in best_params.items():
        print(f"  {key}: {value}")
    
    # Train final model with best params
    print("\n" + "=" * 60)
    print("TRAINING FINAL MODEL WITH BEST PARAMETERS")
    print("=" * 60)
    
    best_model_params = {k: v for k, v in best_params.items() 
                         if k not in ['msle', 'train_time']}
    
    # 🔥 FIX: Asegurar que los parámetros finales también son int
    best_model_params['n_estimators'] = int(best_model_params['n_estimators'])
    best_model_params['num_leaves'] = int(best_model_params['num_leaves'])
    best_model_params['max_depth'] = int(best_model_params['max_depth'])
    
    final_model = LGBMRegressor(
        objective="regression",
        reg_alpha=0.0,
        reg_lambda=0.0,
        verbosity=-1,
        random_state=42,
        **best_model_params
    )
    
    final_model.fit(X_train_prep, y_train_log)
    
    # Final predictions
    valid_pred_log = final_model.predict(X_valid_prep)
    valid_pred = np.expm1(valid_pred_log)
    valid_pred = np.clip(valid_pred, 0, None)
    
    msle_final = mean_squared_log_error(y_valid, valid_pred)
    msle_baseline = mean_squared_log_error(y_valid, np.zeros_like(y_valid))
    
    print(f"\n✓ Final Model MSLE: {msle_final:.6f}")
    print(f"  Baseline MSLE: {msle_baseline:.6f}")
    print(f"  Improvement: {((msle_baseline - msle_final) / msle_baseline * 100):.2f}%")
    
    # Update models dict for submission
    models = {"regressor": final_model}
    
    # Save results to CSV
    results_df.to_csv("grid_search_results.csv", index=False)
    print(f"\n✓ Results saved to: grid_search_results.csv")

GRID SEARCH - SINGLE LGBM REGRESSOR

Total combinations: 729
Estimated time: ~1458 minutes (aprox 2 min/model)

[1/10] Testing: {'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 127, 'max_depth': -1, 'subsample': 0.7, 'colsample_bytree': 0.7}
  MSLE: 0.180962 | Time: 6.3s

[2/10] Testing: {'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 127, 'max_depth': -1, 'subsample': 0.7, 'colsample_bytree': 0.8}
  MSLE: 0.180962 | Time: 6.3s

[2/10] Testing: {'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 127, 'max_depth': -1, 'subsample': 0.7, 'colsample_bytree': 0.8}
  MSLE: 0.181132 | Time: 6.0s

[3/10] Testing: {'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 127, 'max_depth': -1, 'subsample': 0.7, 'colsample_bytree': 0.9}
  MSLE: 0.181132 | Time: 6.0s

[3/10] Testing: {'n_estimators': 400, 'learning_rate': 0.03, 'num_leaves': 127, 'max_depth': -1, 'subsample': 0.7, 'colsample_bytree': 0.9}
  MSLE: 0.181049 | Time: 5.9s

[4/10] Testing: {'n_estimators': 

## Generate Submission

In [ ]:
print("Generating test predictions...")

dd_test = dd.read_parquet(TEST_PATH, engine='pyarrow')
existing_big_cols_test = [c for c in IGNORE_BIG_COLS if c in dd_test.columns]
dd_test = dd_test.drop(columns=existing_big_cols_test)

delayed_parts = dd_test.to_delayed()
print(f"Processing {len(delayed_parts)} test chunks...")

pred_dfs = []
feature_cols_final = X_train_prep.columns.tolist()

for i, d in enumerate(delayed_parts):
    if (i + 1) % 10 == 0:
        print(f"  Chunk {i+1}/{len(delayed_parts)}...")
    
    part_df = d.compute()
    part_df = reduce_memory(part_df)
    
    row_ids = part_df["row_id"].values
    
    # 🔥 IMPORTANTE: Solo seleccionar columnas que existen en test
    available_cols = [c for c in feature_cols_final if c in part_df.columns]
    X_part = part_df[available_cols].copy()
    
    # 🔥 Añadir columnas faltantes con valores por defecto
    for col in feature_cols_final:
        if col not in X_part.columns:
            if col in num_cols:
                X_part[col] = 0  # Numeric → 0
            else:
                X_part[col] = "unknown"  # Categorical → unknown
    
    # Reordenar columnas para que coincidan con el train
    X_part = X_part[feature_cols_final]
    
    # 🔥 IMPORTANTE: Pasar el scaler para normalización consistente
    X_part_prep = preprocess_new(X_part, num_cols, cat_cols, X_train_prep, scaler=scaler)
    
    # Predict based on model version
    if MODEL_VERSION == "single":
        part_pred_log = models["regressor"].predict(X_part_prep)
        part_pred = np.expm1(part_pred_log)
        part_pred = np.clip(part_pred, 0, None)
        
    elif MODEL_VERSION == "two_step":
        buyer_prob = models["classifier"].predict_proba(X_part_prep)[:, 1]
        revenue_pred_log = models["regressor"].predict(X_part_prep)
        revenue_pred = np.expm1(revenue_pred_log)
        revenue_pred = np.clip(revenue_pred, 0, None)
        part_pred = buyer_prob * revenue_pred
    
    pred_dfs.append(pd.DataFrame({
        "row_id": row_ids,
        "iap_revenue_d7": part_pred
    }))
    
    del part_df, X_part, X_part_prep, row_ids, part_pred
    gc.collect()

# Combine and save
submission = pd.concat(pred_dfs, ignore_index=True)
output_file = f"submission_{MODEL_VERSION}.csv"
submission.to_csv(output_file, index=False)

print(f"\n✓ Submission saved: {output_file}")
print(f"Shape: {submission.shape}")
print(f"Sample:\n{submission.head()}")

# Validation checks
print(f"\nValidation checks:")
print(f"  NaN values: {submission.isna().sum().sum()}")
print(f"  Negative values: {(submission['iap_revenue_d7'] < 0).sum()}")

Generating test predictions...
Processing 96 test chunks...
  Chunk 10/96...
  Chunk 10/96...
  Chunk 20/96...
  Chunk 20/96...


KeyboardInterrupt: 

## Summary

**To switch models**: Change `MODEL_VERSION` in the first code cell to:
- `"single"` - Single LightGBM regressor (direct approach)
- `"two_step"` - Classifier + Regressor (probabilistic approach)

**To adjust data size**: Change `TRAIN_SAMPLE_FRAC` (default 0.10 = 10%)

Then re-run all cells!